## Correspondencia de Características (Match)

In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [2]:
# Cargamos la imagen a buscar
img1 = cv.imread('imagenes/mask_libro.png', cv.IMREAD_GRAYSCALE)

# Cargamos la imagen de búsqueda
img2 = cv.imread('imagenes/libro_busqueda_1.jpg', cv.IMREAD_GRAYSCALE)

#### 1. Generamos las características con ORB

In [4]:
# Inicializamos el detector ORB
orb = cv.ORB_create(nfeatures=3000)

# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

print(len(kp1))
print(len(kp2))

1908
3000


#### 1.1. Coincidimos con Fuerza Bruta

In [6]:
# Cremos el objeto BFMatcher
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

# Coincidimos descriptores.
matches = bf.match(des1, des2)

# Los ordenamos según distancia
matches = sorted(matches, key = lambda x:x.distance)

# Dibujamos las primeras 30 coincidencias
n=10
img3 = cv.drawMatches(img1,kp1,img2,kp2,matches[:n],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img3),plt.show()

(<matplotlib.image.AxesImage at 0x1f038752388>, None)

#### 2. Generamos las características con SIFT

In [ ]:
# Creamos el vector de características SIFT
sift = cv.xfeatures2d.SIFT_create()

# Y buscamos según el algoritmo...
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

print(len(kp1))
print(len(kp2))

#### 2.1 Coincidimos con Fuerza Bruta y verificación de proporción (1er & 2do vecino cercano)

In [ ]:
# BFMatcher con parámetros por defecto
bf = cv.BFMatcher()
# K es la cantidad de "best matches" para cada descriptor
matches = bf.knnMatch(des1, des2, k=2)

# Aplicar test de proporción
good = []
for m, n in matches:
    if m.distance < 0.8*n.distance:
        good.append(m)

# Los ordenamos según distancia
good = sorted(good, key = lambda x:x.distance)
# Dibujamos las primeras N coincidencias
N = 30

# cv.drawMatchesKnn espera una lista de listas como coincidencias
img3 = cv.drawMatchesKnn(img1, kp1, img2, kp2, [good[:N]], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv.DrawMatchesFlags_DEFAULT)
plt.figure()
plt.imshow(img3),plt.show()

#### 2.2 Coincidimos con FLANN (Fast Library for Approximate Nearest Neighbors)

Colección de algoritmos optimizados para búsqueda rápida de vecinos cercanos.

In [ ]:
# parámetros FLANN
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # o pasar un diccionario vacío
flann = cv.FlannBasedMatcher(index_params,search_params)
# K es la cantidad de "best matches" para cada descriptor
matches = flann.knnMatch(des1, des2, k=2)

# Necesidad de dibujar solo las coincidencias buenas, se crea una máscara
matchesMask = [[0,0] for i in range(len(matches))]

# Ratio de verificación como figura en el paper de Lowe
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv.DrawMatchesFlags_DEFAULT)
img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
plt.imshow(img3,),plt.show()

#### 2.3 Coincidimos con FLANN + Homografía

In [ ]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)

# Guardar todos las buenas coincidencias según la verificación de Ratio de Lowe
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
print(len(good))

In [ ]:
# https://docs.opencv.org/4.x/d9/dab/tutorial_homography.html

MIN_MATCH_COUNT = 8

if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts, M)
    img2 = cv.polylines(img2, [np.int32(dst)], True, 255, 3, cv.LINE_AA)
else:
    print( "No se encontraron suficientes coincidencias - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

print(mask)

In [ ]:
draw_params = dict(matchColor = (0,255,0), # dibujar coincidencias en verde
                   singlePointColor = None,
                   matchesMask = matchesMask, # dibujar solo inliers
                   flags = 2)
img3 = cv.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
plt.imshow(img3, 'gray'),plt.show()